##### Multi-Way Decomposition from scratch

These are the simplest cpd and Tucker decomposition for nd-arrays


Input: N-order tensor 𝜒𝜖 ℝ^(I1,I2,… ,In) , tensor rank R
Output: coefficients 𝜆n for n=1:N, factor matrices 𝐴^n 𝜖 ℝ^(In,Rn) for n=1:N

Step 1: Initialize: randomly initialize 𝐴^N for n=1:N

repeat
for n = 1, …, N do
𝑇_n = 𝐴(1)^T 𝐴(1) ∗ … ∗ 𝐴(n-1)^T 𝐴(n-1) ∗ 𝐴(n+1)^T 𝐴(n+1) ∗ … ∗ 𝐴(N)^T 𝐴(N)
𝐴(n) = 𝜒(n)(𝐴(N) ⊙ … ⊙ 𝐴(n+1) ⊙ 𝐴(n-1) ⊙ … ⊙ 𝐴(1))𝑇n
end for
until the convergence criterion is satisfied

In [82]:
from scipy.linalg import khatri_rao
import tensorly as tl


def cpd(tensor, rank=2, max_iter = 100, epsilon=1e-10):   
    N = len(tensor.shape) # order of tensor X

    # initialise factor matrices randomly
    A = []
    for n in range(N):
        In =tensor.shape[n]
        A.append (np.random.rand(rank, In))
    # factorise   
    
    print ("A initialised : ")
    print (A)
    factors = []
    prev_factors = []
    for epoch in range(max_iter): # to exit when values are too close, or for loop for max iterations or both  
        prev_factors = factors
        factors = []
        for n in range(N): # for each mode
            print ("mode " + str(n) + " / " + str(N))
            In =tensor.shape[n]
            Tn = np.ones((rank, In))
            print ("Tn initialised :" )
            print (Tn)
            A_Temp = np.ones((rank, In))
            print ("A_Temp initialised : ")
            print (A_Temp)
            cnt = 0
            for j in range(N): # for each other mode
                if n != j: # skip the current mode to ALS
                    if cnt == 0:
                        Tn = np.dot(np.transpose(A[j]), A[j])
                        A_Temp = A[j]
                    else:
                        Tn = np.dot (Tn, np.dot(np.transpose(A[j]), A[j]))
                        A_Temp = khatri_rao (A_Temp, A[j])
                    cnt += 1
                    print ("Tn shape")
                    print (Tn.shape) 
                    
                    print ("An shape")
                    print (A_Temp.shape) # (4, 12)
                    #A_Temp =  tl.tenalg.khatri_rao([A_Temp, A[j]])
                    #print ("tensorly khatri_rao shape")
                    #print (A_Temp.shape)

            Xn = tl.unfold(tensor, n).T
            print ("Xn shape")
            print (Xn.shape)# (12, 12) # X{n} in the pseudo-code
            print ("Tn shape")
            print (Tn.shape) # (2, 12) # T{n} in the pseudo-code
            print ("A_Temp shape")
            print (A_Temp.shape)# (4, 12)
            #print ("np.matmul(Tn, np.transpose(A_Temp)).shape") 
            #print (np.matmul(Tn, np.transpose(A_Temp)).shape)  # (2, 4)
            fn = np.matmul(Xn.T , A_Temp.T)  # (12, 4)
            print ("fn shape")
            print (fn.shape)# (12, 4)
            fn1 = np.matmul(fn.T , Tn.T)  # (4, 4)
            print ("fn1 shape")
            print (fn1.shape)# (4, 4)
            A[n] = fn1 # # A{n} in the pseudo-code
            factors.append(A[n]) 
        factors = np.array(factors)
        if prev_factors != []:    
            if np.all(factors- prev_factors) > 1 - epsilon:            
                return np.array(factors)
                           
                           
    return np.array(factors)       

In [83]:
tensor = tl.tensor([[ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
                        [ 0.,  0.,  0.,  0.,  1.,  1.,  1.,  1.,  0.,  0.,  0.,  0.],
                        [ 0.,  0.,  0.,  0.,  1.,  1.,  1.,  1.,  0.,  0.,  0.,  0.],
                        [ 0.,  0.,  0.,  0.,  1.,  1.,  1.,  1.,  0.,  0.,  0.,  0.],
                        [ 0.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  0.],
                        [ 0.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  0.],
                        [ 0.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  0.],
                        [ 0.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  0.],
                        [ 0.,  0.,  0.,  0.,  1.,  1.,  1.,  1.,  0.,  0.,  0.,  0.],
                        [ 0.,  0.,  0.,  0.,  1.,  1.,  1.,  1.,  0.,  0.,  0.,  0.],
                        [ 0.,  0.,  0.,  0.,  1.,  1.,  1.,  1.,  0.,  0.,  0.,  0.],
                        [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.]])

In [84]:
from tensorly.decomposition import parafac
factors = parafac(tensor, rank=2)
len(factors)

2

In [85]:
[f.shape for f in factors[1]]

[(12, 2), (12, 2)]

In [86]:
factors

(weights, factors) : rank-2 CPTensor of shape (12, 12) 

In [88]:
factors[1]

[array([[ 0.        ,  0.        ],
        [ 1.66009908, -1.11537933],
        [ 1.66009908, -1.11537933],
        [ 1.66009908, -1.11537933],
        [ 3.02615419,  0.9178185 ],
        [ 3.02615419,  0.9178185 ],
        [ 3.02615419,  0.9178185 ],
        [ 3.02615419,  0.9178185 ],
        [ 1.66009908, -1.11537933],
        [ 1.66009908, -1.11537933],
        [ 1.66009908, -1.11537933],
        [ 0.        ,  0.        ]]),
 array([[ 0.        ,  0.        ],
        [ 0.22767585,  0.33886631],
        [ 0.22767585,  0.33886631],
        [ 0.22767585,  0.33886631],
        [ 0.41502477, -0.27884483],
        [ 0.41502477, -0.27884483],
        [ 0.41502477, -0.27884483],
        [ 0.41502477, -0.27884483],
        [ 0.22767585,  0.33886631],
        [ 0.22767585,  0.33886631],
        [ 0.22767585,  0.33886631],
        [ 0.        ,  0.        ]])]

In [89]:
factors_np = cpd(tensor)

A initialised : 
[array([[0.99333161, 0.69627597, 0.99877916, 0.28527412, 0.94292971,
        0.74749978, 0.8453798 , 0.14960632, 0.54132329, 0.76942799,
        0.11487806, 0.87395244],
       [0.5734491 , 0.28012376, 0.04355871, 0.07284214, 0.811406  ,
        0.48722278, 0.0842931 , 0.02350154, 0.58130385, 0.84407316,
        0.46583392, 0.78752491]]), array([[0.8700054 , 0.97074754, 0.51294765, 0.8785119 , 0.26058485,
        0.99848198, 0.57672965, 0.17502783, 0.6195872 , 0.9689963 ,
        0.04558024, 0.33012827],
       [0.80203406, 0.71679606, 0.01624175, 0.7497075 , 0.40896612,
        0.202037  , 0.52053424, 0.71321159, 0.94552957, 0.08315949,
        0.39127079, 0.02677935]])]
mode 0 / 2
Tn initialised :
[[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]]
A_Temp initialised : 
[[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]]
Tn shape
(12, 12)
An shape
(2, 12)
Xn shape
(12, 12)
Tn shape
(12, 12)
A_Temp shape
(2, 12)
fn

<ipython-input-82-53a15eb767a6>:68: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.
  if prev_factors != []:


In [90]:
factors_np.shape

(2, 2, 12)

In [91]:
factors_np[1]

array([[2.32908174e+61, 2.37332120e+61, 7.95189835e+60, 2.27569136e+61,
        9.11205052e+60, 1.76360718e+61, 1.52985900e+61, 1.16712363e+61,
        2.13255230e+61, 1.56860053e+61, 5.64115455e+60, 5.32443012e+60],
       [1.95226363e+61, 1.98934566e+61, 6.66537445e+60, 1.90751118e+61,
        7.63782761e+60, 1.47827622e+61, 1.28234575e+61, 9.78296718e+60,
        1.78753035e+61, 1.31481936e+61, 4.72848190e+60, 4.46299977e+60]])

In [92]:
factors_np[0]

array([[8.22894828e+19, 8.38525203e+19, 2.80950897e+19, 8.04031313e+19,
        3.21940579e+19, 6.23105321e+19, 5.40519053e+19, 4.12359939e+19,
        7.53458426e+19, 5.54206940e+19, 1.99309317e+19, 1.88119031e+19],
       [6.89760095e+19, 7.02861659e+19, 2.35496336e+19, 6.73948476e+19,
        2.69854369e+19, 5.22294188e+19, 4.53069409e+19, 3.45644937e+19,
        6.31557689e+19, 4.64542757e+19, 1.67063407e+19, 1.57683578e+19]])